# Getting the dependencies ready

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" trackio \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth trackio
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

In [ ]:
%%capture
!pip install -qqq fastapi uvicorn requests
!git clone https://github.com/yogesh1801/OpenEnv.git > /dev/null 2>&1
%cd OpenEnv
!git checkout yogesh-julia-env

In [ ]:
import subprocess, sys, os
from pathlib import Path
sys.path.insert(0, './src')
working_directory = str(Path.cwd().parent.absolute() / "OpenEnv")

In [ ]:
from envs.ruby_env import REnv
from envs.ruby_env.models import RubyAction, RubyObservation

# Making sure the Env is up and working

In [ ]:
port = "8003"
localhost = f"http://localhost:{port}"
# openenv_process = subprocess.Popen(
#     [sys.executable, "-m", "uvicorn", "envs.ruby_env.server.app:app", "--host", "0.0.0.0", "--port", port],
#     env = {**os.environ,
#          "PYTHONPATH": f"{working_directory}/src",
#          },
#     stdout = subprocess.PIPE,
#     stderr = subprocess.PIPE,
#     text = True,
#     cwd = working_directory,
# )


# used docker images instead

In [ ]:
import requests
import time
print(requests.get(f"{localhost}/health", timeout = 2).content)
openenv_process = RubyEnv(base_url = localhost)

b'{"status":"healthy"}'


In [ ]:
action = RubyAction(
    core_code = """
def add_nums(a, b)
  a + b
end
""",
    test_code = """
require 'test/unit'

class TestAddNums < Test::Unit::TestCase
  def test_add_positive_numbers
    assert_equal(8, add_nums(5, 3))
  end

  def test_add_negative_numbers
    assert_equal(-8, add_nums(-5, -3))
  end

  def test_add_zero
    assert_equal(5, add_nums(5, 0))
  end

  def test_add_mixed_signs
    assert_equal(2, add_nums(5, -3))
  end
end
"""
)

result = openenv_process.step(action)
result

StepResult(observation=RObservation(done=False, reward=None, metadata={}, stdout='\n══ Testing tmpp0uq0ogl.R ═══════════════════════════════════════════════════════\n\n[ FAIL 0 | WARN 0 | SKIP 0 | PASS 0 ]\n[ FAIL 0 | WARN 0 | SKIP 0 | PASS 1 ]\n[ FAIL 0 | WARN 0 | SKIP 0 | PASS 2 ]\n[ FAIL 0 | WARN 0 | SKIP 0 | PASS 3 ] Done!\n', stderr='', exit_code=0, tests_passed=3, tests_failed=0, code_compiles=True), reward=8, done=False)

#  Getting the model ready with unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2000
lora_rank = 10
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    fast_inference = True,
    load_in_4bit = False,
    max_seq_length = max_seq_length,
    gpu_memory_utilization = 0.85,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 10-27 12:54:35 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.


[2025-10-27 12:54:38] INFO vllm_utils.py:752: Unsloth: Patching vLLM


INFO 10-27 12:54:38 [vllm_utils.py:694] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker'
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 191.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 84.85%
Unsloth: Your GPU has CUDA compute capability 9.4 with VRAM = 191.69 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1500. Num Sequences = 400.
Unsloth: vLLM's KV Cache can use up to 156.48 GB. Also swap space = 6 GB.
Unsloth: Not an error,

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-27 12:54:44 [default_loader.py:314] Loading weights took 2.52 seconds
INFO 10-27 12:54:44 [punica_selector.py:20] Using PunicaWrapperGPU.
INFO 10-27 12:54:45 [gpu_model_runner.py:2917] Model loading took 7.0195 GiB and 3.044069 seconds
INFO 10-27 12:54:50 [backends.py:609] Using cache directory: /root/.cache/vllm/torch_compile_cache/9f1bce1ea9/rank_0_0/backbone for vLLM's torch.compile
INFO 10-27 12:54:50 [backends.py:623] Dynamo bytecode transform time: 4.72 s
INFO 10-27 12:54:51 [backends.py:207] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.320 s
INFO 10-27 12:55:06 [monitor.py:34] torch.compile takes 6.04 s in total
INFO 10-27 12:55:06 [gpu_worker.py:337] Available KV cache memory: 153.07 GiB
INFO 10-27 12:55:07 [kv_cache_utils.py:1229] GPU KV cache size: 1,433,040 tokens
INFO 10-27 12:55:07 [kv_cache_utils.py:1234] Maximum concurrency for 1,500 tokens per request: 952.82x
INFO 10-27 12:55:07 [vllm_utils.py:699] Unsloth: Running patched vLLM v

...

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:07<00:00, 18.27it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:06<00:00, 17.27it/s]

INFO 10-27 12:55:20 [gpu_model_runner.py:3843] Graph capturing finished in 13 secs, took 0.83 GiB
INFO 10-27 12:55:20 [vllm_utils.py:706] Unsloth: Patched vLLM v1 graph capture finished in 13 secs.


INFO 10-27 12:55:21 [core.py:238] init engine (profile, create kv cache, warmup model) took 35.71 seconds
INFO 10-27 12:55:21 [llm.py:343] Supported tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['q_norm', 'input_layernorm', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'norm1', 'k_norm', 'post_feedforward_layernorm', 'attention_norm', 'post_layernorm', 'ffn_norm', 'layer_norm1', 'layer_norm2', 'norm2']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Just some info: will skip parsing ['q_norm', 'input_layernorm', 'pre_feedforward_layernorm', 'cross_attn_post_attention_layernorm', 'post_attention_layernorm', 'norm1', 'cross_attn_input_layernorm', 'k_norm', 'post_feedforward_layernorm', 'attention_norm', 'post_layernorm', 'ffn_norm', 'layer_norm1', 'layer_norm2', 'norm2']


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.10.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Finetuning starts here


In [ ]:
Ruby_code_gen_prompt = """
You are a precise and pragmatic Ruby programmer.
Write a **single Ruby function** that correctly solves the problem described below.

Rules:
- The code must be syntactically correct and runnable as is.
- Use standard Ruby function syntax:
    def function_name(args)
      ...
    end
- Use only base Ruby (no external libraries or gems).
- Do **not** wrap the code in any scaffolding, classes, or a `main` function.
- Do **not** include any test code, print statements, or comments.
- Do **not** hardcode specific test cases or outputs — the function must work for general inputs.
- The **function name must exactly match** the one used in the provided tests.
- Return only the Ruby function and nothing else (no explanations, no extra text).
- Pay attention to correct Ruby syntax: use `=`, `.to_i`, `.to_f`, `.to_s` for conversions.
- Arrays and strings in Ruby are 0-indexed.
- Ensure the function works correctly for general input cases.

Test Reference (for context only, do not include in the output):
{ruby_test}

Code:
""".strip()


In [ ]:
import re
def remove_ticks(text):
    text = re.sub(r'^```ruby\s*\n?', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n?```\s*$', '', text)

    return text

In [ ]:
def ruby_env_reward(completions, **kwargs):
    rewards = []
    test_codes_list = kwargs.get('ruby_test', [])

    for i, completion in enumerate(completions):
        response = completion[0]["content"]
        core_code = remove_ticks(response)
        test_code = test_codes_list[i]
        try:
            result = openenv_process.reset()
            action = RubyAction(core_code=core_code, test_code=test_code)
            result = openenv_process.step(action)

            reward = result.reward if result.reward is not None else 0.0

        except Exception as e:
            print(f"Error for completion {i}:", e)
            reward = 0.0

        rewards.append(reward)

    return rewards

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_parquet("../ruby_trainset.parquet")

df = df[["ruby_prompt", "ruby_test", "first_test_case"]]
dataset = Dataset.from_pandas(df)

dataset = dataset.map(lambda x: {
    "prompt": [
        {"role" : "system", "content": Ruby_code_gen_prompt.format(ruby_test=x["first_test_case"])},
        {"role" : "user", "content": x["ruby_prompt"]},
    ],
})
print(dataset[0])

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

{'r_prompt': 'Implement a function `echo_nums(x, y)` that takes two integers, `x` and `y`, and returns a vector of all numerical values within the range from `x` to `y`, inclusive. The function should handle cases where `x` is greater than `y` by returning an empty vector.', 'r_test': 'library(testthat)\ntest_that("echo_nums returns correct sequence when x is less than y", {\n  expect_equal(echo_nums(1, 3), c(1, 2, 3))\n})\ntest_that("echo_nums returns correct sequence when x is equal to y", {\n  expect_equal(echo_nums(5, 5), c(5))\n})\ntest_that("echo_nums returns empty vector when x is greater than y", {\n  expect_equal(echo_nums(10, 5), numeric(0))\n})', 'prompt': [{'content': 'You are a precise and pragmatic R programmer.\nWrite a **single R function** that correctly solves the problem described below.\nRules:\n- The code must be syntactically correct and runnable as is.\n- Use traditional R function syntax with proper braces and return statements.\n- Use only base R (no external p

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    temperature = 1.0,
    learning_rate = 5e-5,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 3,
    max_prompt_length = 2048,
    max_completion_length = 1024,
    num_train_epochs = 1,
    max_steps = 100,
    save_steps = 250,
    report_to = "trackio",
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 3


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    remove_unused_columns = False,
    reward_funcs = [ruby_env_reward],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128004}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,224 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 1 x 1) = 3
 "-____-"     Trainable parameters = 15,196,160 of 3,227,945,984 (0.47% trained)


* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/.cache/huggingface/trackio


* Created new run: graceful-canyon-26
WARNING 10-27 12:55:43 [processor.py:228] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / r_env_reward / mean,rewards / r_env_reward / std
1,0.000000,3.000000,5.000000,92.333336,67.000000,111.000000,0.000000,92.333336,67.000000,111.000000,0,0,0,0,0,0.000491,3.000000,5.000000
2,0.000000,1.666667,5.507571,73.333336,18.000000,113.000000,0.000000,73.333336,18.000000,113.000000,No Log,No Log,No Log,No Log,No Log,0.000000,1.666667,5.507571
3,0.000000,13.333334,4.618802,128.000000,123.000000,135.000000,0.000000,128.000000,123.000000,135.000000,No Log,No Log,No Log,No Log,No Log,0.000170,13.333333,4.618802
4,0.000000,3.000000,7.810250,85.000000,63.000000,122.000000,0.000000,85.000000,63.000000,122.000000,No Log,No Log,No Log,No Log,No Log,0.000389,3.000000,7.810250
5,0.000000,-1.666667,0.577350,55.333336,49.000000,61.000000,0.000000,55.333336,49.000000,61.000000,No Log,No Log,No Log,No Log,No Log,0.000607,-1.666667,0.577350
6,0.000000,4.333333,5.507571,52.333336,41.000000,68.000000,0.000000,52.333336,41.000000,68.000000,No Log,No Log,No Log,No Log,No Log,0.000307,4.333333,5.507571
7,0.000000,3.333333,4.509250,50.000000,30.000000,63.000000,0.000000,50.000000,30.000000,63.000000,No Log,No Log,No Log,No Log,No Log,0.000257,3.333333,4.509250
8,0.000000,5.666667,13.279056,118.333336,66.000000,218.000000,0.000000,118.333336,66.000000,218.000000,No Log,No Log,No Log,No Log,No Log,0.000365,5.666667,13.279057
9,0.000000,2.000000,5.196153,38.000000,32.000000,47.000000,0.000000,38.000000,32.000000,47.000000,No Log,No Log,No Log,No Log,No Log,0.000332,2.000000,5.196152
10,0.000000,1.666667,2.309401,84.000000,78.000000,94.000000,0.000000,84.000000,78.000000,94.000000,No Log,No Log,No Log,No Log,No Log,0.000448,1.666667,2.309401


Unsloth: Will smartly offload gradients to save VRAM!
* Run finished. Uploading logs to Trackio (please wait...)


TrainOutput(global_step=100, training_loss=1.5094738033418764e-05, metrics={'train_runtime': 293.9707, 'train_samples_per_second': 1.021, 'train_steps_per_second': 0.34, 'total_flos': 0.0, 'train_loss': 1.5094738033418764e-05})

# Inference and post training

In [ ]:
model.save_lora("grpo_saved_lora_ruby")

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_parquet("../ruby_testset.parquet")

df = df[["ruby_prompt", "ruby_test", "task_id"]]
dataset = Dataset.from_pandas(df)

MAX_LEN = 2000

dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": Ruby_code_gen_prompt.format(ruby_test=x["first_test_case"])[:MAX_LEN]},
        {"role": "user", "content": x["ruby_prompt"][:MAX_LEN]},
    ],
})

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

results = []

for i in range(len(dataset)):
    task_id = dataset[i]["task_id"]
    ruby_test = dataset[i]["ruby_test"]
    message = dataset[i]["prompt"]
    message[1]

    text = tokenizer.apply_chat_template(
        message,
        add_generation_prompt=True,
        tokenize=False,
    )

    response = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_saved_lora_ruby"),
    )

    ruby_code = response[0].outputs[0].text
    ruby_code = remove_ticks(ruby_code)

    results.append({
        "task_id": task_id,
        "ruby_code": ruby_code,
        "ruby_test": ruby_test
    })

result_df = pd.DataFrame(results)
result_df.to_parquet("ruby_code_output_smaller_model.parquet", index=False)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%|                                                                                      …

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …